In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-alloydb-pg-python/blob/main/samples/langchain_quick_start.ipynb)

---
# **Introduction**

In this codelab, you'll learn how to create a powerful interactive generative AI application using Retrieval Augmented Generation powered by [AlloyDB for PostgreSQL](https://cloud.google.com/alloydb) and [LangChain](https://www.langchain.com/). We will be creating an application grounded in a [Netflix Movie dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), allowing you to interact with movie data in exciting new ways.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic python knowledge

## What you'll learn

* How to deploy a AlloyDB for PostgreSQL instance
* How to use AlloyDB for PostgreSQL as a VectorStore
* How to use AlloyDB for PostgreSQL as a DocumentLoader
* How to use AlloyDB for PostgreSQL for ChatHistory storage

## What you'll need
* A Google Cloud Account and Google Cloud Project
* A web browser such as [Chrome](https://www.google.com/chrome/)

# **Setup and Requirements**

In the following instructions you will learn to:
1. Install required dependencies for our application
2. Set up authentication for our project
3. Set up a AlloyDB for PostgreSQL Instance
4. Import the data used by our application

## Install dependencies
First you will need to install the dependencies needed to run this demo app.

In [ ]:
%pip install langchain_google_alloydb_pg

%pip install langchain langchain-google-vertexai

In [ ]:
%pip install "google-cloud-alloydb-connector[pg8000]"

## Authenticate to Google Cloud within Colab
In order to access your Google Cloud Project from this notebook, you will need to Authenticate as an IAM user.

In [ ]:
from google.colab import auth

auth.authenticate_user()

## Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook so that you can leverage Google Cloud from within Colab.

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = ""  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

## Enable APIs for AlloyDB and Vertex AI within your project

You will need to enable these APIs in order to create an AlloyDB database and utilize Vertex AI as an embeddings service!

In [ ]:
# enable GCP services
!gcloud services enable alloydb.googleapis.com
!gcloud services enable aiplatform.googleapis.com

## Set up AlloyDB
You will need a **Postgres** AlloyDB instance for the following stages of this notebook. Please set the following variables.

In [ ]:
# @markdown Please fill in the both the Google Cloud region and name of your AlloyDB instance. Once filled in, run the cell.

# Please fill in these values.
region = ""  # @param {type:"string"}
instance_name = ""  # @param {type:"string"}
database_name = ""  # @param {type:"string"}
password = input("Please provide a password to be used for 'postgres' database user: ")
cluster_name = ""  # @param {type:"string"}

### Create an AlloyDB Instance
If you have already created an AlloyDB Cluster and Instance, you can skip these steps and skip to the connectivity section.

First let's create an AlloyDB Cluster.
> ⏳ - Creating an AlloyDB cluster may take a few minutes.


In [ ]:
# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide the name of your database_name"

#create the AlloyDB Cluster
!gcloud beta alloydb clusters create {cluster_name} --password={password} --region={region}



Now that we have created our AlloyDB Cluster, we can create an instance attached to our cluster with the following command.
> ⏳ - Creating an AlloyDB instance may take a few minutes.

In [ ]:
!gcloud beta alloydb instances create {instance_name} --instance-type=PRIMARY --cpu-count=2 --region={region} --cluster={cluster_name}

In order to connect to your newly created AlloyDB instance from this notebook, you will need to enable public IP on your instance. Alternatively, you can follow [these instructions](https://cloud.google.com/alloydb/docs/connect-external) to connect to an AlloyDB for PostgreSQL instance with Private IP from outside your VPC.

In [ ]:
!gcloud beta alloydb instances update {instance_name} --region={region} --cluster={cluster_name} --assign-inbound-public-ip=ASSIGN_IPV4

Now let's set the connection string that we will use to connect to our instance.

In [ ]:
connection_string = "projects/{0}/locations/{1}/clusters/{2}/instances/{3}".format(
    project_id, region, cluster_name, instance_name
)
print(connection_string)

In [ ]:
from google.cloud.alloydb.connector import Connector, IPTypes
import sqlalchemy

# initialize Connector object
connector = Connector()


# function to return the database connection
def getconn():
    conn = connector.connect(
        connection_string,
        "pg8000",
        user="postgres",
        password=password,
        db="postgres",
        enable_iam_auth=False,
        ip_type=IPTypes.PUBLIC,
    )
    return conn


# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://", creator=getconn, isolation_level="AUTOCOMMIT"
)

##Create a Database

Now that we can connect to our AlloyDB instance from this notebook, let's create a database we will use to store the data for this application. You may get an error that there is no public ip address, this is because assigning a public ip address takes a few minutes. Please wait and retry this step if you hit an error!

In [ ]:
create_db_cmd = sqlalchemy.text(
    f"CREATE DATABASE {database_name}",
)
with pool.connect() as db_conn:
    db_conn.execute(create_db_cmd)
connector.close()

## Import data to your database

Now that you have your database, you will need to import data! We will be using a [Netflix Dataset from Kaggle](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here is what the data looks like:

| show_id | type    | title                | director         | cast                                                                                                                                                  | country       | date_added        | release_year | rating | duration  | listed_in                                    | description                                                                                                                                                                           |
|---------|---------|----------------------|------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|-------------------|--------------|--------|-----------|----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| s1      | Movie   | Dick Johnson Is Dead | Kirsten Johnson  |                                                                                                                                                        | United States | September 25, 2021 | 2020         | PG-13  | 90 min    | Documentaries                                | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.                              |
| s2      | TV Show | Blood & Water        |                  | Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng | South Africa  | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, TV Dramas, TV Mysteries | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.                                   |
| s3      | TV Show | Ganglands            | Julien Leclercq  | Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera                                   |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Crime TV Shows, International TV Shows, TV Action & Adventure | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.                                     |
| s4      | TV Show | Jailbirds New Orleans |                  |                                                                                                                                                        |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Docuseries, Reality TV                        | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.                                   |
| s5      | TV Show | Kota Factory         |                  | Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar                                                 | India        | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, Romantic TV Shows, TV Comedies | In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. |


Instead of leaving it to you to figure out how to insert these documents, we have prepared code to help you insert the csv into your AlloyDB for PostgreSQL database.

First lets download the csv file and add it to our notebook.

In [ ]:
!gsutil cp gs://cloud-samples-data/langchain/first_five_netflix_titles.csv .

We have downloaded the csv file to our home directory, to see it run ls. You should also be able to see the csv file populate in the "Files" tab.

In [ ]:
!ls

Lets add a connection function to connect to our db using the AlloyDB sychronous connector.

In [ ]:
from google.cloud.alloydb.connector import Connector, IPTypes
import sqlalchemy

# initialize Connector object
connector = Connector()


# function to return the database connection
def getconn():
    conn = connector.connect(
        connection_string,
        "pg8000",
        user="postgres",
        password=password,
        db=database_name,
        enable_iam_auth=False,
        ip_type=IPTypes.PUBLIC,
    )
    return conn


# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://", creator=getconn, isolation_level="AUTOCOMMIT"
)

In this next step we will
1. Create the table into which we will insert the data.
2. Map over the columns of our csv file to the columns of our datatable and insert the data!

In [ ]:
import pandas as pd

create_table_cmd = sqlalchemy.text(
    'CREATE TABLE netflix_titles ( \
    show_id VARCHAR, \
    type VARCHAR, \
    title VARCHAR, \
    director VARCHAR, \
    "cast" VARCHAR, \
    country VARCHAR, \
    date_added VARCHAR, \
    release_year INTEGER, \
    rating VARCHAR, \
    duration VARCHAR, \
    listed_in VARCHAR, \
    description TEXT \
    )',
)

netflix_data = "/content/first_five_netflix_titles.csv"

df = pd.read_csv(netflix_data)
insert_data_cmd = sqlalchemy.text(
    """
    INSERT INTO netflix_titles VALUES (:show_id, :type, :title, :director,
      :cast, :country, :date_added, :release_year, :rating,
      :duration, :listed_in, :description)
    """
)

parameter_map = [
    {
        "show_id": row["show_id"],
        "type": row["type"],
        "title": row["title"],
        "director": row["director"],
        "cast": row["cast"],
        "country": row["country"],
        "date_added": row["date_added"],
        "release_year": row["release_year"],
        "rating": row["rating"],
        "duration": row["duration"],
        "listed_in": row["listed_in"],
        "description": row["description"],
    }
    for index, row in df.iterrows()
]

with pool.connect() as db_conn:
    db_conn.execute(create_table_cmd)
    db_conn.execute(
        insert_data_cmd,
        parameter_map,
    )
    db_conn.commit()
connector.close()

# **Use case 1: AlloyDB for Postgres as a document loader**

---



Now that you have data in your database, you are ready to use AlloyDB for PostgreSQL as a document loader. This means we will pull data from the database and load it into memory as documents. We can then feed these documents into the vector store.

Next let's connect to our AlloyDB for PostgreSQL instance using the AlloyDBEngine class.

In [ ]:
from langchain_google_alloydb_pg import AlloyDBEngine, Column, AlloyDBLoader

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)

Once we initialize an AlloyDBEngine object, we can pass it into the AlloyDBSQLLoader to connect to a specific database. As you can see we also pass in a query, table_name and a list of columns. The query tells the loader what query to use to pull data. The "content_columns" argument refers to the columns that will be used as "content" in the document object we will later construct. The rest of the columns in that table will become the "metadata" associated with the documents.

In [ ]:
table_name = "netflix_titles"
content_columns = ["title", "director", "cast", "description"]
loader = await AlloyDBLoader.create(
    engine=engine,
    query=f"SELECT * FROM {table_name};",
    content_columns=content_columns,
)

Next let's define a function "collect_async_items" to asynchronously pull documents from our database.

In [ ]:
async def collect_async_items(docs_generator):
    """Collects items from an async generator."""
    docs = []
    async for doc in docs_generator:
        docs.append(doc)
    return docs

Then let's run the function to pull our documents from out database using our document loader. You can see the first 5 documents from the database here. Nice, you just used AlloyDB Postgres as a document loader!

In [ ]:
documents = await collect_async_items(loader.alazy_load())
print(f"Loaded {len(documents)} from the database. 5 Examples:")
for doc in documents[:5]:
    print(doc)

# **Use case 2: AlloyDB for PostgreSQL as Vector Store**

Now, let's learn how to put all of the documents we just loaded into a vector store so that we can use vector search to answer our user's questions!

### Create Your Vector Store table

Based on the documents that we loaded before, we want to create a table with a vector column as our vector store. We will start it by intializing a vector table by calling the `init_vectorstore_table` function from our `engine`. As you can see we list all of the columns for our metadata. We also specify a vector size, 768, that corresponds with the length of the vectors computed by the model our embeddings service uses, Vertex AI's textembedding-gecko.


In [ ]:
from langchain_google_alloydb_pg import AlloyDBEngine, Column

sample_vector_table_name = "movie_vector_table_samples"

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)

engine.init_vectorstore_table(
    sample_vector_table_name,
    vector_size=768,
    metadata_columns=[
        Column("show_id", "VARCHAR", nullable=True),
        Column("type", "VARCHAR", nullable=True),
        Column("country", "VARCHAR", nullable=True),
        Column("date_added", "VARCHAR", nullable=True),
        Column("release_year", "INTEGER", nullable=True),
        Column("rating", "VARCHAR", nullable=True),
        Column("duration", "VARCHAR", nullable=True),
        Column("listed_in", "VARCHAR", nullable=True),
    ],
    overwrite_existing=True,  # Enabling this will recreate the table if exists.
)

### Try inserting the documents into the vector table

Now we will create a vector_store object backed by our vector table in the AlloyDB database. Let's load the data from the documents to the vector table. Note that for each row, the embedding service will be called to compute the embeddings to store in the vector table. Pricing details can be found [here](https://cloud.google.com/vertex-ai/pricing).

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_alloydb_pg import AlloyDBVectorStore, AlloyDBEngine

# Initialize the embedding service. In this case we are using version 003 of Vertex AI's textembedding-gecko model. In general, it is good practice to specify the model version used.
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=project_id
)

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)

vector_store = AlloyDBVectorStore.create_sync(
    engine=engine,
    embedding_service=embeddings_service,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

Now let's try to put the documents data into the vector table. Here is a code example to load the first 5 documents in the list.

In [ ]:
import uuid

docs_to_load = documents[:5]

# ! Uncomment the following line to load all 8,800+ documents to the database vector table with calling the embedding service.
# docs_to_load = documents

ids = [str(uuid.uuid4()) for i in range(len(docs_to_load))]
vector_store.add_documents(docs_to_load, ids)

### Import the rest of your data into your vector table

You don't have to call the embedding service 8,800 times to load all the documents for the demo. Instead, we have prepared a csv with the all 8,800+ rows with pre-computed embeddings in a csv file. Again, let's import the csv using gsutil.

In [ ]:
!gsutil cp gs://cloud-samples-data/langchain/alloydb/netflix_titles_embeddings.csv .

And now let's insert the csv data into the table containing our vectors.

In [ ]:
netflix_data = "/content/langchain_alloydb_netflix_computed_embeddings.csv"
df = pd.read_csv(netflix_data)
insert_data_cmd = sqlalchemy.text(
    """
    INSERT INTO movie_vector_table_samples VALUES (:langchain_id, :content, :embedding, :show_id,
      :type, :country, :date_added, :release_year, :rating,
      :duration, :listed_in, :langchain_metadata)
    """
)

parameter_map = [
    {
        "langchain_id": row["langchain_id"],
        "content": row["content"],
        "embedding": row["embedding"],
        "show_id": row["show_id"],
        "type": row["type"],
        "country": row["country"],
        "date_added": row["date_added"],
        "release_year": row["release_year"],
        "rating": row["rating"],
        "duration": row["duration"],
        "listed_in": row["listed_in"],
        "langchain_metadata": row["langchain_metadata"],
    }
    for index, row in df.iterrows()
]

with pool.connect() as db_conn:
    db_conn.execute(
        insert_data_cmd,
        parameter_map,
    )
    db_conn.commit()
connector.close()

# **Use case 3: AlloyDB for PostgreSQL as Chat Memory**

Next we will add chat history (called “memory” in the context of LangChain) to our application so the LLM can retain context and information across multiple interactions, leading to more coherent and sophisticated conversations or text generation. We can use AlloyDB for PostgreSQL as “memory” storage in our application so that the LLM can use context from prior conversations to better answer the user’s prompts. First let's initialize AlloyDB for PostgreSQL as memory storage.

In [ ]:
from langchain_google_alloydb_pg import AlloyDBChatMessageHistory, AlloyDBEngine

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)
message_table_name = "message_store"

engine.init_chat_history_table(table_name=message_table_name)

chat_history = AlloyDBChatMessageHistory.create_sync(
    engine,
    session_id="my-test-session",
    table_name=message_table_name,
)

Here is an example of how you would add a user message and how you would add an AI message.

In [ ]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

# **Conversational RAG Chain backed by AlloyDB**

So far we've tested out using AlloyDB for PostgreSQL as document loader, Vector Store and Chat Memory. Now let's use it in the `ConversationalRetrievalChain`.

We will build a chat bot that can answer movie related questions based on the vector search results.

First let's initialize all of our AlloyDBEngine object to use as a connection in our vector store and chat_history.

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_google_alloydb_pg import (
    AlloyDBEngine,
    AlloyDBVectorStore,
    AlloyDBChatMessageHistory,
)

# Intialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=project_id
)

# Intialize the engine
engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)
# Intialize the Vector Store
vector_store = AlloyDBVectorStore.create_sync(
    engine=engine,
    embedding_service=embeddings_service,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

# Intialize the AlloyDBChatMessageHistory
chat_history = AlloyDBChatMessageHistory.create_sync(
    engine,
    session_id="my-test-session",
    table_name="message_store",
)

Let's create a prompt for the LLM. Here we can add instructions specific to our application, such as "Don't make things up".

In [ ]:
# Prepare some prompt templates for the ConversationalRetrievalChain
prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

Now let's use our vector store as a retreiver. Retreiver's in Langchain allow us to literally "retrieve" documents.

In [ ]:
# Intialize retriever, llm and memory for the chain
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

Now let's intialize our LLM, in this case we are using Vertex AI's "gemini-pro".

In [ ]:
llm = VertexAI(model_name="gemini-pro", project=project_id)

We clear our chat history, so that our application starts without any prior context to other conversations we have had with the application.

In [ ]:
chat_history.clear()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

Now let's create a conversational retrieval chain. This will allow the LLM to use chat history in it's responses, meaning we can ask it follow up questions to our questions instead of having to start from scratch after each inquiry.

In [ ]:
# create the ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# ask some questions
q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Jonny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

# browser the chat history
chat_history.messages